# Como buscar informações da API do Spotify

Este notebook mostra a forma que consegui puxar as informações para meu projeto. Há outras possibilidades também de buscas e no final mostro algumas referências utilizadas para isso.

Para fazer as buscas é necessário as chaves de desevolvedor para API, sendo aprovado a requisição, você obtém as duas chaves como abaixo.

In [1]:
## instalando a biblioteca do spotipy
# pip install spotipy

# importando as bibliotecas
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# criando um objeto para as chaves de acesso na API
cid = 'YOUR_KEY'
secret = 'YOUR_SECRET_KEY'

# obtendo acesso à API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

Atualmente a API do spotify limita até 2000 querys, ou seja, como no exemplo abaixo para fazer a busca por anos, cada ano retornará 2000 linhas, tenho a opção de colocar os dois anos juntos, porém só virá as 2000 e assim por diante, então para isso usei um loop para puxar e com isso me retornará 4000 linhas, sendo realizado as buscas de cada vez.

O tempo para as buscas podem demorar dependendo do que você está buscando.

In [ ]:
# inicia o contador de tempo
start = time.time()

# cria as listas vazias para receber as informações buscadas
artist_name = []
track_name = []
popularity = []
track_id = []

# cria uma lista para o loop nas consultas
years = ['year:2016','year:2017']

# executando o loop
for year in years:
    
    # encontra os elementos da busca
    for i in range(0,2000,50):
        track_results = sp.search(q=year, type='track', limit=50, offset=i, market='BR')
        
        # realiza o loop buscando as informações e inserindo nas listas vazias
        for i, t in enumerate(track_results['tracks']['items']):
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            popularity.append(t['popularity'])

# finaliza a contagem do tempo e mostra a duração da busca
end = time.time()
print(end - start)

A busca acima são dos elementos principais para formar nosso conjunto de dados, abaixo podemos buscar mais atributos.

In [ ]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    

    # features
    track_id = features[0]['id']
    mode = features[0]['mode']
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    valence = features[0]['valence']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    duration_ms = features[0]['duration_ms']
    time_signature = features[0]['time_signature']
    

    track = [name, album, artist, release_date, length, popularity, track_id, mode, acousticness, danceability, 
             energy, instrumentalness, liveness, valence, loudness, speechiness, tempo, duration_ms, time_signature]
    
    return track

A célula de cima, estrutura as informações para buscar por elas. A célula de baixo, roda um loop coletando as informações dos atributos definidos acima e por fim coloca em dataframe e salva em arquivo *.csv*.

**Obs.:** esse próximo loop é demorado dependendo da quantidade de elementos buscados e atenção ao *time.sleep* antes de iniciar cada rodada do loop, pois se não colocarmos esse tempo o servidor nos derruba.

In [ ]:
start = time.time()

# loop over track ids 
tracks = []
for i in range(len(track_id)):
    track = getTrackFeatures(track_id[i])
    tracks.append(track)
    time.sleep(2)

# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'track_id', 'mode', 
                                     'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence', 
                                     'loudness', 'speechiness', 'tempo', 'duration_ms', 'time_signature'])
df.to_csv("spotify_2016-2017.csv", sep = ',')

end = time.time()
print(end - start)

## Para buscar a generos pelos artistas

A API não nos dá como padrão da *consulta por músicas* os generos dela, portanto a solução que encontrei está abaixo.<br>
Pode ser criado na sequencia, porém como tinha pensado nisso depois, segue minha solução. Importei a base das músicas por ano e coloquei a coluna dos *artistas* em uma lista. e coloquei essa lista no loop, extraindo as informações necessárias.

In [ ]:
df_2016_2017_genre = pd.read_csv('dados/spotify_2016-2017_.csv')

artist_16_17 = pd.read_csv('dados/spotify_2016-2017.csv')['artist'].tolist()

In [ ]:
start = time.time()
genre_artist = []

for i in artist_16_17:
#     result = sp.search(sp.track(track_id[i])['album']['artists'][0]['name'])
    result = sp.search(i)
    track = result['tracks']['items'][0]

    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
#     print("artist genres:", artist["genres"])
    genre_artist.append(artist["genres"])
    time.sleep(2)
    
end = time.time()
print(end - start)

#     album = sp.album(track["album"]["external_urls"]["spotify"])
#     print("album genres:", album["genres"])
#     print("album release-date:", album["release_date"])

Concatenei a lista dos generos junto com a lista que já havia puxado e salvei ela local novamente.

In [ ]:
# pd.DataFrame(genre_artist)
data_transposed = zip(genre_artist)
df = pd.DataFrame(data_transposed, columns=['genre'])

In [ ]:
df_2016_2017_genre = pd.concat([df_2017_2017_genre, df], axis=1).drop(['Unnamed: 0'], axis=1)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('spotify_2016_2017_.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_2016_2017_genre.to_excel(writer, sheet_name='Sheet1', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

## Para buscar a playlist de usuários

Necessário ter o `username` e o `id` da playlist. Uma das formas é entrar no site do spotify logado e buscar pelo perfil da pessoa, no barra de endereços, dentro da playlist o *id* poderá ser encontrado.

Depois o processo é semelhante do que já foi descrito acima.

In [ ]:
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('login.usuario', '2x1Nwheedt0LZcMo8etCzZ')

In [ ]:
print(len(ids))
print(ids)

In [ ]:
sp.audio_features(ids[0])

In [10]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    track_id = features[0]['id']
    mode = features[0]['mode']
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    valence = features[0]['valence']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    duration_ms = features[0]['duration_ms']
    time_signature = features[0]['time_signature']

    track = [name, album, artist, release_date, length, popularity, track_id, mode, acousticness, danceability, 
             energy, instrumentalness, liveness, valence, loudness, speechiness, tempo, duration_ms, time_signature]
    
    return track

In [ ]:
start = time.time()

# loop over track ids 
tracks = []
for i in range(len(ids)):
    time.sleep(.5)
    track = getTrackFeatures(ids[i])
    tracks.append(track)

# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'track_id', 'mode', 
                                     'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence', 
                                     'loudness', 'speechiness', 'tempo', 'duration_ms', 'time_signature'])
# df.to_csv("jwserafim_playlist.csv", sep = ',')

end = time.time()
print(end - start)

## Considerações

Para saber quais informações buscar, deve-se ter um conhecimento prévio de *web scraping*, isso é devido ao resultado que a API nos retorna sendo semelhante as estruturas de página web. A célula abaixo mostra um exemplo.

In [2]:
artist= 'Gusttavo lima'
# track= 'tipo borboleta'
track= 'trevo tu'

# track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')
track_id = sp.search(q=' track:' + track, type='track')
# for i in track_id['tracks']['items'][0]:
#     print(i)

id_track = track_id['tracks']['items'][0]['id']
track_id['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1sPg5EHuQXTMElpZ4iUgXe'},
  'href': 'https://api.spotify.com/v1/artists/1sPg5EHuQXTMElpZ4iUgXe',
  'id': '1sPg5EHuQXTMElpZ4iUgXe',
  'name': 'ANAVITÓRIA',
  'type': 'artist',
  'uri': 'spotify:artist:1sPg5EHuQXTMElpZ4iUgXe'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4iWkwAVzssjb8XgxdoOL6M'},
  'href': 'https://api.spotify.com/v1/artists/4iWkwAVzssjb8XgxdoOL6M',
  'id': '4iWkwAVzssjb8XgxdoOL6M',
  'name': 'Tiago Iorc',
  'type': 'artist',
  'uri': 'spotify:artist:4iWkwAVzssjb8XgxdoOL6M'}]

In [3]:
track_id['tracks']['items'][0]['external_urls']

{'spotify': 'https://open.spotify.com/track/2vRBYKWOyHtFMtiK60qRz7'}

## Referencias

https://morioh.com/p/31b8a607b2b0<br>
https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b<br>
https://developer.spotify.com/dashboard/applications/85bde5058f48488eb76c9a41fd7942eb